In [1]:
import pandas as pd
from pathlib import Path
import cbpro
import itertools
import time 
import matplotlib.pyplot as plt
import datetime as datetime
from datetime import datetime
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import statistics
from plotly.subplots import make_subplots
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from dash import Dash, html, dcc
import dash_leaflet as dl
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
client = cbpro.PublicClient()
client.get_product_trades(product_id='ETH-USD')

symbol = 'ETH-USD'

def trades ():
    get_trades = client.get_product_trades(product_id=f'{symbol}')
    
    df = list(itertools.islice(get_trades, 15000))
    
    df = pd.DataFrame(df[:-1])
    
    df['price'] = pd.to_numeric(df['price'])
    df['size'] = pd.to_numeric(df['size'])

    df['trade_dollar_size'] = df['price'] * df['size']
    df ['change_in_price'] = df['price'].diff()

    df ['change_in_size'] = df['size'].diff()

    df['trade_dollar_size'] = pd.to_numeric(df['price'])
    df['change_in_price'] = pd.to_numeric(df['size'])
    df['change_in_size'] = pd.to_numeric(df['price'])

    df[['price', 'size','trade_dollar_size','change_in_price']].describe()

    df.drop(['trade_id'], axis=1)
    
    df.loc[df['side'] == 'buy', 'size'] = df['size'] * -1
    df.loc[df['side'] == 'sell', 'size'] = df['size'] * 1
    
#     df.to_csv(r"D:\Trading\Bot\Docs/ETHUSD Trades.csv",index=False)
    
#     df = pd.read_csv("D:\Trading\Bot\Docs/ETHUSD Trades.csv")
    
    df['time'] = df['time'].values.astype('datetime64[s]')
    
    df_5m = df.resample('5min', on='time').agg({'price':'mean', 'size':'sum','side':'count'}).rename(columns={'price':'average price','size': 'sum of size','side':'volume'})

    df_5m['change_in_price'] = df_5m['average price'].diff()
    df_5m['change_in_size'] = df_5m['sum of size'].diff()
    df_5m['change_in_volume'] = df_5m['volume'].diff()

    df_5m.to_csv(r"D:\Trading\Bot\Docs/ETHUSD Trades_5m.csv",index=True)
    
    df_5m = pd.read_csv(r"D:\Trading\Bot\Docs/ETHUSD Trades_5m.csv")
    
    return(df_5m)

In [3]:
df_5m = trades()
df_5m

,time,average price,sum of size,volume,change_in_price,change_in_size,change_in_volume
0,2022-11-24 05:15:00,1199.632140,-32.380960,458,NaN,NaN,NaN
1,2022-11-24 05:20:00,1201.382188,-95.152865,1732,1.750048,-62.771905,1274.0
2,2022-11-24 05:25:00,1203.315467,-14.810124,1509,1.933279,80.342741,-223.0
3,2022-11-24 05:30:00,1202.928532,81.374798,1369,-0.386935,96.184922,-140.0
4,2022-11-24 05:35:00,1201.753974,38.803232,1404,-1.174557,-42.571566,35.0
5,2022-11-24 05:40:00,1202.494333,51.348586,1020,0.740359,12.545354,-384.0
6,2022-11-24 05:45:00,1203.583001,-32.045259,1043,1.088668,-83.393845,23.0
7,2022-11-24 05:50:00,1202.483444,-83.887217,1089,-1.099557,-51.841958,46.0
8,2022-11-24 05:55:00,1200.804118,-84.893471,1615,-1.679326,-1.006254,526.0
9,2022-11-24 06:00:00,1200.102240,59.064317,1460,-0.701878,143.957788,-155.0


In [4]:
# while True:
#     if (datetime.now().minute % 5 == 0) & (datetime.now().second == 0) :
#         df_5m = trades()
#         break
#         print(df_5m)

In [5]:
# def trades_5m():
#     df = pd.read_csv("D:\Trading\Bot\Docs/ETHUSD Trades.csv")
    
#     df['time'] = df['time'].values.astype('datetime64[s]')
    
#     df_5m = df.resample('5min', on='time').agg({'price':'mean', 'size':'sum','side':'count'}).rename(columns={'price':'average price','size': 'sum of size','side':'volume'})

#     df_5m['change_in_price'] = df_5m['average price'].diff()
#     df_5m['change_in_size'] = df_5m['sum of size'].diff()
#     df_5m['change_in_volume'] = df_5m['volume'].diff()

#     df_5m.to_csv(r"D:\Trading\Bot\Docs/ETHUSD Trades_5m.csv",index=True)
    
#     return(df_5m)

In [6]:
app = dash.Dash(__name__)

color = '#F3ECB0',

app.layout = html.Div(
   [ html.Div([
        html.H1('Live Update'),
        html.Div(id='live-update-text'),
        dcc.Graph(id='fig_v',
                  responsive = True),
        dcc.Interval(
            id='fig_1_update',
            interval=1*5000, # in milliseconds
            n_intervals=0
        )
    ]),
    
       html.Div([
        html.Div(id='live-update-text'),
        dcc.Graph(id='fig_p',
                  responsive = True),
        dcc.Interval(
            id='fig_2_update',
            interval=1*5000, # in milliseconds
            n_intervals=0
        )
    ])]
)

In [7]:
@app.callback(Output('fig_v','figure'),
              Output('fig_p', 'figure'),
              Input('fig_1_update', 'n_intervals'))
def update_graph_live(n):
    
    df_5m = trades()
    df_5m = df_5m.round(2)
    
#     fig = px.scatter(df_5m, x='sum of size',
#                      y='average price',
#                      text='change_in_price',
#                      size='volume',color='time',
#                      template="plotly_dark",
#                      title='Size v Price')
#     fig.update_layout(autosize=True)
#     fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='Gray')
    
    fig_v = px.scatter(df_5m, x='volume',y='average price', text = 'change_in_price', color='time',template="plotly_dark",size='volume',title='Volume Distribution & Average Price')
    
    fig_p = px.scatter(df_5m, x='time',
                     y='average price',
                     trendline="ols",
                     text='sum of size',
                     size='volume',
                     color='time',
                     template="plotly_dark",
                     title='Price v Change Time')
    fig_v.update_layout(plot_bgcolor='#212121',paper_bgcolor='#212121')
    fig_v.update_yaxes(showgrid=True, gridwidth=1, gridcolor='Gray')
    fig_v.update_xaxes(showgrid=True, gridwidth=1, gridcolor='Gray')
    
    
    fig_p.update_layout(plot_bgcolor='#212121',paper_bgcolor='#212121')
    fig_p.update_yaxes(showgrid=True, gridwidth=1, gridcolor='Gray')
    fig_p.update_xaxes(showgrid=True, gridwidth=1, gridcolor='Gray')
    
#     fig = go.Figure(go.Waterfall(x=df_5m['time'],
#                              y=df_5m['change_in_price'],
#                              orientation = 'v',
#                              textposition = 'outside', 
#                              text = df_5m['sum of size'],
#                              hovertext = ['x','y'],
#                              decreasing = {"marker":{"line":{"color":"red", "width":2}}},
#                              increasing = {"marker":{"color":"Green"}},
#                          ))
#     fig.update_layout(autosize=True)
    
#     fig.add_trace(go.Bar(x=df_5m['time'], y=df_5m['volume']))

#     fig.append_trace({
#         'x': df_5m['time'],
#         'y': df_5m['change_in_price'],
#         'name': 'Volume',
#         'mode': 'lines+markers',
#         'type': 'scatter'
#     }, 1, 1)

    return (fig_v, fig_p)

if __name__ == '__main__':
    app.run_server(debug=True,use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [8]:
# df = pd.read_csv("D:\Trading\Bot\Docs/ETHUSD Trades.csv")

In [9]:
# # df['time'] = pd.to_datetime(df['time'],utc=True)
# df['time'] = df['time'].values.astype('datetime64[s]')

In [10]:
# df_5m = df.resample('5min', on='time').agg({'price':'mean', 'size':'sum','side':'count'}).rename(columns={'price':'average price','size': 'sum of size','side':'volume'})

# df_5m['change_in_price'] = df_5m['average price'].diff()
# df_5m['change_in_size'] = df_5m['sum of size'].diff()
# df_5m['change_in_volume'] = df_5m['volume'].diff()

# df_5m.to_csv(r"D:\Trading\Bot\Docs/ETHUSD Trades_5m.csv",index=True)

In [11]:
# df_5m = pd.read_csv('D:\Trading\Bot\Docs/ETHUSD Trades_5m.csv')

# fig = px.scatter(df_5m, x='sum of size',y='average price',size='volume',color='time', title='Average Price v Size')
# fig.update_layout(autosize=True)
# fig.show()

In [12]:
# fig = px.scatter(df_5m, x='volume',y='average price', text = 'sum of size', color='time',size='volume',title='Volume Distribution & Average Price')
# fig.update_layout(autosize=True)
# fig.show()

In [13]:
# df_5m = trades()

# fig = px.bar(df_5m, x='time', y='volume',color='average price',text='average price',title='Time & Volume Distribution Bars')
# fig.update_layout(autosize=True)
# fig.show()

In [14]:
# fig = px.scatter(df_5m, x='change_in_size',y='change_in_price',text='sum of size',size='volume',color='time', title='Change in Price & Change in Price')
# fig.update_layout(autosize=True)
# fig.show()

In [15]:
# df_5m.tail(20)

In [16]:
# df_5m = df_5m.tail(10).round(2)

# fig = go.Figure(go.Waterfall(x=df_5m['time'],
#                              y=df_5m['change_in_price'],
#                              orientation = 'v',
#                              textposition = 'outside', 
#                              text = df_5m['sum of size'],
#                              hovertext = ['x','y'],
#                              decreasing = {"marker":{"line":{"color":"red", "width":2}}},
#                              increasing = {"marker":{"color":"Green"}},
#                          ))
# fig.update_layout(autosize=True)
# fig.show()

In [17]:
# app = Dash(__name__)

# app.layout = html.Div(children=[
#     html.H1(children='Waterfall Graph'),

#     html.Div(children='''
#         A guide graph
#     '''),

#     dcc.Graph(
#         id='example-graph',
#         figure=fig
#     )
# ])

In [18]:
# if __name__ == '__main__':
#     app.run_server(debug=True,use_reloader=False)